In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os

C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encoun

In [2]:
epochs = 250

In [3]:
from mtg_jamendo_dataset.scripts import commons

input_file = 'C:/Projects/Music_Tagging/mtg_jamendo_dataset/data/autotagging_moodtheme.tsv'
tracks, tags, extra = commons.read_file(input_file)

Reading: 18486 tracks, 4506 albums, 1533 artists


In [4]:
paths = []
tags = []
for song_id, song_data in tracks.items():
    paths.append(song_data['path'])
    tags.append(song_data['mood/theme'])

df = pd.DataFrame({'paths': paths, 'tags': tags})
df

,paths,tags
0,48/948.mp3,{background}
1,50/950.mp3,{background}
2,51/951.mp3,{background}
3,65/2165.mp3,{film}
4,63/2263.mp3,{melancholic}
...,...,...
18481,56/1422056.mp3,"{advertising, dramatic, epic, movie}"
18482,57/1422057.mp3,"{advertising, dramatic, epic, movie}"
18483,58/1422058.mp3,"{dramatic, epic, movie}"
18484,59/1422059.mp3,"{advertising, dramatic, epic, movie}"


In [5]:
df['tags'] = df['tags'].astype(str)

# Remove curly braces, split the tags, and strip whitespaces and quotes
df['tags'] = df['tags'].str.replace('{', '').str.replace('}', '').str.split(',')
df['tags'] = df['tags'].apply(lambda tags: [tag.strip().strip("'").strip('"').strip() for tag in tags])

# Get unique tags
unique_tags = set()
for tags in df['tags']:
    unique_tags.update(tags)

# Convert the set of unique tags to a list
unique_classes = list(unique_tags)

# Print or use the unique tags
print(unique_classes)

['calm', 'background', 'adventure', 'corporate', 'nature', 'movie', 'soundscape', 'fun', 'groovy', 'documentary', 'christmas', 'inspiring', 'summer', 'uplifting', 'fast', 'mellow', 'upbeat', 'funny', 'party', 'dramatic', 'deep', 'slow', 'energetic', 'powerful', 'retro', 'advertising', 'epic', 'trailer', 'meditative', 'hopeful', 'holiday', 'sport', 'children', 'cool', 'commercial', 'sad', 'motivational', 'ambiental', 'soft', 'sexy', 'film', 'positive', 'ballad', 'space', 'emotional', 'drama', 'relaxing', 'romantic', 'love', 'melancholic', 'travel', 'dream', 'horror', 'action', 'heavy', 'melodic', 'happy', 'game', 'dark']


C:\Users\athar\AppData\Local\Temp\ipykernel_13288\3162749104.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['tags'] = df['tags'].str.replace('{', '').str.replace('}', '').str.split(',')


In [6]:
def data_table(tracks):
    paths = []
    tags = []
    for song_id, song_data in tracks.items():
        paths.append(song_data['path'])
        tags.append(song_data['mood/theme'])

    df = pd.DataFrame({'paths': paths, 'tags': tags})
    df['tags'] = df['tags'].astype(str)
    df['tags'] = df['tags'].str.replace('{', '').str.replace('}', '')
    df['paths'] = df['paths'].str.replace('/', '_')
    df['paths'] = 'C:/Projects/Music_Tagging/large_mel_spec/' + df['paths']
    df['paths'] = df['paths'].str.replace('.mp3', '.low.npy')

    return df

In [7]:
df = data_table(tracks)
df.head(10)

C:\Users\athar\AppData\Local\Temp\ipykernel_13288\1584220965.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['tags'] = df['tags'].str.replace('{', '').str.replace('}', '')
C:\Users\athar\AppData\Local\Temp\ipykernel_13288\1584220965.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['paths'] = df['paths'].str.replace('.mp3', '.low.npy')


,paths,tags
0,C:/Projects/Music_Tagging/large_mel_spec/48_94...,'background'
1,C:/Projects/Music_Tagging/large_mel_spec/50_95...,'background'
2,C:/Projects/Music_Tagging/large_mel_spec/51_95...,'background'
3,C:/Projects/Music_Tagging/large_mel_spec/65_21...,'film'
4,C:/Projects/Music_Tagging/large_mel_spec/63_22...,'melancholic'
5,C:/Projects/Music_Tagging/large_mel_spec/46_33...,"'calm', 'melodic'"
6,C:/Projects/Music_Tagging/large_mel_spec/47_33...,"'calm', 'melodic'"
7,C:/Projects/Music_Tagging/large_mel_spec/48_33...,"'calm', 'melodic'"
8,C:/Projects/Music_Tagging/large_mel_spec/49_33...,"'calm', 'melodic'"
9,C:/Projects/Music_Tagging/large_mel_spec/50_33...,"'calm', 'melodic'"


In [8]:
df['tags'] = df['tags'].str.replace("'", "").str.split(', ')

mlb = MultiLabelBinarizer()

# Transform the 'tags' column into binary columns
binary_labels = mlb.fit_transform(df['tags'])

# Create a new DataFrame with binary labels
df_labels = pd.DataFrame(binary_labels, columns=mlb.classes_)

# Concatenate the new labels DataFrame with the original DataFrame
df = pd.concat([df, df_labels], axis=1)

# Drop the original 'tags' column if you no longer need it
df.drop('tags', axis=1, inplace=True)

df.head()

,paths,action,adventure,advertising,ambiental,background,ballad,calm,children,christmas,...,slow,soft,soundscape,space,sport,summer,trailer,travel,upbeat,uplifting
0,C:/Projects/Music_Tagging/large_mel_spec/48_94...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,C:/Projects/Music_Tagging/large_mel_spec/50_95...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,C:/Projects/Music_Tagging/large_mel_spec/51_95...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C:/Projects/Music_Tagging/large_mel_spec/65_21...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C:/Projects/Music_Tagging/large_mel_spec/63_22...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from tensorflow.keras.utils import Sequence

class SpectrogramDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size, mel_shape=(128, 1024, 1), spectrogram_dir='path_to_spectrograms'):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.mel_shape = mel_shape
        self.spectrogram_dir = spectrogram_dir

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = (index + 1) * self.batch_size
        batch_df = self.dataframe.iloc[start_idx:end_idx]
        
        X = np.zeros((len(batch_df), *self.mel_shape))
        y = batch_df.iloc[:, 1:].values  # Assuming binary class labels are in columns 1 onwards

        for i, spectrogram_path in enumerate(batch_df['paths']):
            spectrogram = np.load(os.path.join(self.spectrogram_dir, spectrogram_path))
            X[i] = spectrogram.reshape(self.mel_shape)

        return X, y

In [10]:
batch_size = 32
mel_shape = (128, 1024, 1)  # Modify this according to your mel spectrogram shape
spectrogram_dir = 'C:/Projects/Music_Tagging/processed_data/'

train_df, val_test_df = train_test_split(df, test_size=0.3, random_state=17)  # Allocating 30% for combined validation and test sets

# Further split the validation/test set into separate validation and test sets
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=17)  # Splitting half for validation and half for test

# Create data generators for training, validation, and test sets
train_generator = SpectrogramDataGenerator(train_df, batch_size=batch_size, mel_shape=mel_shape, spectrogram_dir=spectrogram_dir)
val_generator = SpectrogramDataGenerator(val_df, batch_size=batch_size, mel_shape=mel_shape, spectrogram_dir=spectrogram_dir)
test_generator = SpectrogramDataGenerator(test_df, batch_size=batch_size, mel_shape=mel_shape, spectrogram_dir=spectrogram_dir)

In [11]:
initial_learning_rate = 0.001

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0000001)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [12]:
base_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=mel_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_df.columns) - 1, activation='sigmoid')  # Output layer with one node per class
])

base_model.summary()

# Compile the model
base_model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss=BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'),tf.metrics.AUC(curve='PR'),tfa.metrics.F1Score(num_classes=59, average='macro')])
# Train the model
base_model_history = base_model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[early_stopping,reduce_lr])

# Evaluate the model
loss, accuracy, rocauc, prauc, f1macro = base_model.evaluate(test_generator)
print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')

base_model.save("base_model.h5")

tf.keras.backend.clear_session()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 1022, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 511, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 509, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 254, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 252, 64)       36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 126, 64)      0

In [19]:
import matplotlib.pyplot as plt

# Train the model and save the history
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[early_stopping, reduce_lr])

# Plot training & validation loss values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Similarly, for other metrics like ROC-AUC, PR-AUC, and F1 Score
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model ROC-AUC')
plt.ylabel('ROC-AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['f1_score'])  # Adjust the key based on how it's named in history
plt.plot(history.history['val_f1_score'])  # Adjust the key based on how it's named in history
plt.title('Model F1 Score')
plt.ylabel('F1 Score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [13]:
# threshold = 0.1

# spectrogram_path = 'C:/Projects/Music_Tagging/processed_data/00_468500.low.npy'
# spectrogram = np.load(spectrogram_path)  # Replace with the path to your spectrogram file
# spectrogram = np.expand_dims(spectrogram, axis=0)  # Add a batch dimension

# predictions = model.predict(spectrogram)

# class_indices = np.where(predictions[0] >= threshold)[0]  # Use a threshold to select classes

# # Get the class labels corresponding to the selected indices
# selected_classes = [unique_classes[i] for i in class_indices]

# # Print or use the selected classes
# print("Selected classes:", selected_classes)

In [18]:
tf.keras.backend.clear_session()

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Dropout, ELU, Flatten
from tensorflow.keras.models import Sequential

class CNN(tf.keras.Model):
    def __init__(self, num_class=len(train_df.columns) - 1):
        super(CNN, self).__init__()

        # init bn
        self.bn_init = BatchNormalization()

        # layer 1
        self.conv_1 = Conv2D(64, 3, padding='same')
        self.bn_1 = BatchNormalization()
        self.mp_1 = MaxPooling2D((2, 4))

        # layer 2
        self.conv_2 = Conv2D(128, 3, padding='same')
        self.bn_2 = BatchNormalization()
        self.mp_2 = MaxPooling2D((2, 4))

        # layer 3
        self.conv_3 = Conv2D(128, 3, padding='same')
        self.bn_3 = BatchNormalization()
        self.mp_3 = MaxPooling2D((2, 4))

        # # layer 4
        self.conv_4 = Conv2D(128, 3, padding='same')
        self.bn_4 = BatchNormalization()
        self.mp_4 = MaxPooling2D((3, 5))

        # # layer 5
        # self.conv_5 = Conv2D(64, 3, padding='same')
        # self.bn_5 = BatchNormalization()
        # self.mp_5 = MaxPooling2D((4, 4))

        # classifier
        self.flatten = Flatten()
        self.dense = Dense(num_class, activation='sigmoid')
        self.dropout = Dropout(0.5)

    def call(self, x):
        # init bn
        x = self.bn_init(x)

        # layer 1
        x = self.mp_1(ELU()(self.bn_1(self.conv_1(x))))

        # layer 2
        x = self.mp_2(ELU()(self.bn_2(self.conv_2(x))))

        # layer 3
        x = self.mp_3(ELU()(self.bn_3(self.conv_3(x))))

        # # layer 4
        x = self.mp_4(ELU()(self.bn_4(self.conv_4(x))))

        # # layer 5
        # x = self.mp_5(ELU()(self.bn_5(self.conv_5(x))))

        # classifier
        x = self.flatten(x)
        x = self.dropout(x)
        return self.dense(x)

# Instantiate and build the model
vgg_model = CNN(num_class=len(train_df.columns) - 1)
vgg_model.build((None, 128, 1024, 1))  # Replace with your input shape, e.g., (None, height, width, channels)

In [17]:
# Compile the model
vgg_model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'),tf.metrics.AUC(curve='PR'),tfa.metrics.F1Score(num_classes=59, average='macro')])
vgg_model.summary()
# Train the model
vgg_model_history = vgg_model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[early_stopping,reduce_lr])

# Evaluate the model
loss, accuracy, rocauc, prauc, f1macro = vgg_model.evaluate(test_generator)
print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')

base_model.save("vgg_model.h5")

tf.keras.backend.clear_session()

Model: "cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  multiple                 4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           multiple                  640       
                                                                 
 batch_normalization_5 (Batc  multiple                 256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           multiple                  73856     
                                                             

ResourceExhaustedError: Graph execution error:

Detected at node 'cnn_1/batch_normalization_5/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance
      app.start()
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packa

In [ ]:
# threshold = 0.1

# spectrogram_path = 'C:/Projects/Music_Tagging/processed_data/00_468500.low.npy'
# spectrogram = np.load(spectrogram_path)  # Replace with the path to your spectrogram file
# spectrogram = spectrogram.reshape(1, 128, 128, 1)  # Add a batch dimension

# predictions = model.predict(spectrogram)

# class_indices = np.where(predictions[0] >= threshold)[0]  # Use a threshold to select classes

# # Get the class labels corresponding to the selected indices
# selected_classes = [unique_classes[i] for i in class_indices]

# # Print or use the selected classes
# print("Selected classes:", selected_classes)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LSTM, Dense, Flatten, TimeDistributed, Input
from tensorflow.keras.models import Model

def create_crnn_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    conv_1 = Conv2D(64, (3, 3), activation='relu')(input_layer)
    pool_1 = MaxPooling2D((2, 2))(conv_1)
    bn_1 = BatchNormalization()(pool_1)

    conv_2 = Conv2D(128, (3, 3), activation='relu')(bn_1)
    pool_2 = MaxPooling2D((2, 2))(conv_2)
    bn_2 = BatchNormalization()(pool_2)

    conv_3 = Conv2D(256, (3, 3), activation='relu')(bn_2)
    pool_3 = MaxPooling2D((2, 2))(conv_3)
    bn_3 = BatchNormalization()(pool_3)

    conv_4 = Conv2D(256, (3, 3), activation='relu')(bn_3)
    pool_4 = MaxPooling2D((2, 2))(conv_4)
    bn_4 = BatchNormalization()(pool_4)
    
    # Reshape output for RNN input
    reshape = TimeDistributed(Flatten())(bn_4)

    # RNN layer
    lstm = LSTM(128, return_sequences=False)(reshape)
    lstm = LSTM(128, return_sequences=False)(reshape)
    # Fully connected layer
    dense = Dense(64, activation='relu')(lstm)
    output_layer = Dense(num_classes, activation='sigmoid')(dense)  # sigmoid for multi-label classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Define your model
input_shape = (128, 1024, 1)  # example input shape, adjust to your needs
num_classes = 59  # adjust to the number of classes
crnn_model = create_crnn_model(input_shape, num_classes)

In [ ]:
# Compile the model
crnn_model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss=BinaryCrossentropy(), metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'),tf.metrics.AUC(curve='PR'),tfa.metrics.F1Score(num_classes=59, average='macro')])
crnn_model.summary()
# Train the model
crnn_model_history = crnn_model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=[early_stopping,reduce_lr])

# Evaluate the model
loss, accuracy, rocauc, prauc, f1macro = crnn_model.evaluate(test_generator)
print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')

base_model.save("crnn_model.h5")

tf.keras.backend.clear_session()

In [ ]:
# spectrogram_path = 'C:/Projects/Music_Tagging/processed_data/00_387400.low.npy'
# spectrogram = np.load(spectrogram_path)  # Replace with the path to your spectrogram file
# spectrogram = spectrogram.reshape(1, 128, 128, 1)  # Add a batch dimension

# predictions = model.predict(spectrogram)

# # Define a threshold
# threshold = 0.1  # Adjust based on your model and problem

# # Apply threshold to prediction to get binary values
# predicted_labels = (predictions > threshold).astype(int)

# # Print or use the predicted labels
# print("Predicted labels:", predicted_labels)